# Reccurent Neural Network for Quarterly Financials

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm import tqdm

## Import Financial Data

In [2]:
datadir = 'Income_SP500/'
filelist = os.listdir(datadir)

In [3]:
feats=['TotalRevenue', 'OperatingRevenue', 'PretaxIncome', 'NetIncomeCommonStockholders', 'NetIncome', 'NetIncomeIncludingNoncontrollingInterests', 'NetIncomeContinuousOperations', 'DilutedNIAvailtoComStockholders', 'NetIncomeFromContinuingAndDiscontinuedOperation', 'NormalizedIncome', 'ReconciledDepreciation', 'NetIncomeFromContinuingOperationNetMinorityInterest', 'TaxRateForCalcs', 'TaxEffectOfUnusualItems']
companies=[]
data=np.empty((len(filelist),len(feats)),dtype=object)
for it,filename in enumerate(filelist):
    companies.append(filename.split('_')[0])
    with open(datadir+filename, 'r') as cfile:
        for line in cfile:
            line=[num.replace(',','') for num in line.strip().split('\"') if num.replace(',','') != '']
            if line[0] in feats:
                data[it][feats.index(line[0])]=np.array([float(num) for num in line[1:]])

## Recurrent Neural Network

In [9]:
if torch.cuda.is_available():
    DEVICE= torch.device('cuda')
else:   
    DEVICE = torch.device('cpu')
class FinRNN(nn.Module):
    def __init__(self, n_features):
        super(FinRNN, self).__init__()
        self.hidden1 = 100
        self.hidden2 = 100

        self.rnn = nn.RNN(n_features, self.hidden1, nonlinearity='relu', batch_first=True)
        self.FC1 = nn.Linear(self.hidden1, self.hidden2)
        self.FC2 = nn.Linear(self.hidden2, 1)
        
    def forward(self, x):
        # set hidden states for RNN layer
        # h0 = torch.zeros(1, x.size(0), self.hidden1).to(DEVICE)
        x, hout = self.rnn(x)
        # Take final RNN output and feed to Fully Connected Layers
        x = self.FC1(x[:, -1, :])
        x = torch.relu(x)
        x = self.FC2(x)
        return x

model = FinRNN(14)
print('# of Parameters:', sum([param.numel() for param in model.parameters()]))

indata = torch.randn(5, 3, 14)

# h0 = torch.randn(2, 3, 20)
output = model(indata)
print(indata.size())
print(output.size())
print(output)

# of Parameters: 21801
torch.Size([5, 3, 14])
torch.Size([5, 1])
tensor([[-0.1079],
        [-0.1148],
        [-0.1387],
        [-0.0791],
        [-0.1252]], grad_fn=<AddmmBackward0>)
